## Optimization TD3 Notebook - Integer Linear Programming

In [33]:
# Introduction

import numpy as np
import pandas as pd
from math import floor

# Chargement des données

V = np.loadtxt("VolumeItems50.txt")
N = len(V)       # nombre d'items
C = 2.7          # capacité des boîtes

In [34]:
# Création de la liste des boites

Indices3 = np.where(V <= C/3)[0]
Indices2 = np.where((V > C/3) & (V <= C/2))[0]
Indices1 = np.where(V > C/2)[0]

Box = []

# Boîtes contenant 3 items
nB3 = floor(len(Indices3) / 3)
for b in range(nB3):
    items = Indices3[3*b:3*(b+1)]
    box = {
        'NumberBox': b+1,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

# Items restants du groupe 3
nI3 = len(Indices3) % 3
if nI3 > 0:
    Indices2 = np.concatenate([Indices2, Indices3[-nI3:]])

# Boîtes contenant 2 items
nB2 = floor(len(Indices2) / 2)
for b in range(nB2):
    noBox = nB3 + b + 1
    items = Indices2[2*b:2*(b+1)]
    box = {
        'NumberBox': noBox,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

# Items restants du groupe 2
nI2 = len(Indices2) % 2
if nI2 > 0:
    Indices1 = np.concatenate([Indices1, Indices2[-nI2:]])
    
# Boîtes contenant 1 item
nB1 = len(Indices1)
for b in range(nB1):
    noBox = nB3 + nB2 + b + 1
    items = np.array([Indices1[b]])
    box = {
        'NumberBox': noBox,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

total_boxes = nB3 + nB2 + nB1
df = pd.DataFrame(data = Box)

# Affichage des résultats
print(df)
print("\nNumber of boxes:", total_boxes)

    NumberBox         Items  NumberItems  UnusedVolume
0           1     [2, 3, 4]            3          1.35
1           2    [5, 8, 13]            3          0.98
2           3  [14, 15, 16]            3          0.99
3           4  [18, 21, 22]            3          1.92
4           5  [24, 25, 26]            3          1.58
5           6  [27, 30, 31]            3          1.06
6           7  [32, 34, 37]            3          1.44
7           8  [38, 39, 40]            3          0.63
8           9  [42, 43, 45]            3          1.84
9          10  [46, 49, 50]            3          0.81
10         11      [10, 19]            2          0.61
11         12      [23, 36]            2          0.74
12         13      [41, 48]            2          0.44
13         14           [1]            1          1.19
14         15           [6]            1          0.48
15         16           [7]            1          0.90
16         17           [9]            1          0.99
17        

In [35]:
# [Question 1] - Remplissage séquentiel des items (heuristique)

# N est le nombre d'items, V est le vecteur des volumes de ces items.
# On fait une boucle sur N: pour chaque item, on regarde d'abord si il reste de
# la place dans la boite précédente (si volume S < capacité C). Si oui, on la 
# remplit et on met à jour son volume (S = S + V(i)). Si non, on prend une nouvelle
# boite et on la remplit. Ainsi de suite jusqu'à avoir paqueté tous les items.


nb_box = 0         # Indice de la boîte utilisée
S = 0              # Volume de la boite actuelle (en cours de remplissage)
i = 0              # Indice de l'objet en cours de paquetage
Box_seq = []       # liste des boîtes

Box_seq.append({'NumberBox': 1, 'Items': []})

while i < N:
    if S + V[i] <= C:
        S += V[i]
        Box_seq[nb_box]['Items'].append(i)
        i += 1
    else:
        Box_seq[nb_box]['NumberItems'] = len(Box_seq[nb_box]['Items'])
        Box_seq[nb_box]['UnusedVolume'] = C - S
        # Ouvrir une nouvelle boîte
        nb_box += 1
        Box_seq.append({'NumberBox': nb_box+1, 'Items': []})
        S = 0

# Finalisation de la dernière boîte
if 'NumberItems' not in Box_seq[nb_box]:
    Box_seq[nb_box]['NumberItems'] = len(Box_seq[nb_box]['Items'])
    Box_seq[nb_box]['UnusedVolume'] = C - S

print(pd.DataFrame(data=Box_seq))
print("\nNumber of used boxes (greedy heuristic) :", len(Box_seq))

    NumberBox             Items  NumberItems  UnusedVolume
0           1         [0, 1, 2]            3          0.01
1           2         [3, 4, 5]            3          0.01
2           3            [6, 7]            2          0.01
3           4            [8, 9]            2          0.01
4           5              [10]            1          1.12
5           6          [11, 12]            2          0.01
6           7      [13, 14, 15]            3          0.99
7           8              [16]            1          0.45
8           9          [17, 18]            2          1.13
9          10      [19, 20, 21]            3          0.29
10         11  [22, 23, 24, 25]            4          0.62
11         12          [26, 27]            2          0.47
12         13          [28, 29]            2          0.13
13         14      [30, 31, 32]            3          0.16
14         15              [33]            1          2.45
15         16              [34]            1          0.

In [36]:
# [Question 2]

# On a un problème linéaire entier (ILP). L'ensemble N des objets est fini,
# on peut donc redéfinir le problème comme binary linear problem, en
# introduisant des variables binaires.

On définit les variables suivantes:

\[
x_{n,b} =
\begin{cases} 
1, & \text{si l'objet } n \text{ est placé dans la boîte } b \\
0, & \text{sinon}
\end{cases}
\]

\[
y_b =
\begin{cases} 
1, & \text{si la boîte } b \text{ est utilisée} \\
0, & \text{sinon}
\end{cases}
\]

On définit également:

\[
\begin{cases} 
v_n & \text{est le volume de l'objet } n,\\
C & \text{est la capacité maximale d'une boîte,}\\
N & \text{est le nombre total d'objets,}\\
B & \text{est la borne supérieure du nombre de boîtes obtenue dans la question 1.}
\end{cases}
\]

On peut donc écrire les contraines et le coût ainsi:

\textbf{C1: Le volume des boites ne doit pas dépasser leur capacité} \\
\text{Pour chaque boite, on somme sur les volumes des objets présents et on vérifie que cela est inférieur à la capacité C} 
\[
\forall b \in \{1, \dots, B\}, \quad \sum_{1 \leq n \leq N} v_n x_{n,b} \leq C y_b
\]

\textbf{C2: Tous les items doivent être dans une boite} \\
\text{Pour chaque item, on vérifie qu'il est au moins placé dans une des boites} 
\[
\forall n \in \{1, \dots, N\}, \quad \sum_{b=1}^{B} x_{n,b} = 1
\]

\textbf{Fonction de coût, à minimiser}

\[
\min \sum_{b=1}^{B} y_b
\]

On a donc

\alpha_n = v_n \\
\beta = C